# DAKTARI- THE AI MEDICAL CHATBOT

# PROJECT SUMMARY

# 1. BUSINESS UNDERSTANDING

# 2. BUSINESS PROBLEM

# 3. OBJECTIVES

## 3.1 Main objective

## 3.2 Specific objective

## 3.3 Research Questions

## 3.4 Metric of success

# 4. DATA UNDERSTANDING

## 4.1 Data Limitation

# 5. DATA EXPLORATION

## 5.1 Loading a Dataset

In [14]:
# import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
# pip install huggingface_hub


In [16]:
df = pd.read_parquet("hf://datasets/DrBenjamin/ai-medical-chatbot/dialogues.parquet")
df.head()

,Description,Patient,Doctor
0,Q. What does abutment of the nerve root mean?,"Hi doctor,I am just wondering what is abutting...",Hi. I have gone through your query with dilige...
1,Q. What should I do to reduce my weight gained...,"Hi doctor, I am a 22-year-old female who was d...",Hi. You have really done well with the hypothy...
2,Q. I have started to get lots of acne on my fa...,Hi doctor! I used to have clear skin but since...,Hi there Acne has multifactorial etiology. Onl...
3,Q. Why do I have uncomfortable feeling between...,"Hello doctor,I am having an uncomfortable feel...",Hello. The popping and discomfort what you fel...
4,Q. My symptoms after intercourse threatns me e...,"Hello doctor,Before two years had sex with a c...",Hello. The HIV test uses a finger prick blood ...


In [17]:
df.shape

(256916, 3)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256916 entries, 0 to 256915
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Description  256916 non-null  object
 1   Patient      256916 non-null  object
 2   Doctor       256916 non-null  object
dtypes: object(3)
memory usage: 5.9+ MB


In [19]:
df.isnull().sum()

,0
Description,0
Patient,0
Doctor,0


Since our data is not missing any missing values we can move to cleaning the text.

## 5.2 Data Cleaning

In [20]:
import nltk # natural language toolkit
import re # regular expressions
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
# Download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### 5.2.1 Text preprocessing

In [22]:
df.columns

Index(['Description', 'Patient', 'Doctor'], dtype='object')

**Tokenization**

Let's breaks down each text into a list of words **tokens**

In [24]:
# Apply tokenization to each text column
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
for col in ['Description', 'Patient', 'Doctor']:
    df[col + '_tokens'] = df[col].astype(str).apply(word_tokenize)

df[['Description_tokens', 'Patient_tokens', 'Doctor_tokens']].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,Description_tokens,Patient_tokens,Doctor_tokens
0,"[Q, ., What, does, abutment, of, the, nerve, r...","[Hi, doctor, ,, I, am, just, wondering, what, ...","[Hi, ., I, have, gone, through, your, query, w..."
1,"[Q, ., What, should, I, do, to, reduce, my, we...","[Hi, doctor, ,, I, am, a, 22-year-old, female,...","[Hi, ., You, have, really, done, well, with, t..."
2,"[Q., I, have, started, to, get, lots, of, acne...","[Hi, doctor, !, I, used, to, have, clear, skin...","[Hi, there, Acne, has, multifactorial, etiolog..."
3,"[Q, ., Why, do, I, have, uncomfortable, feelin...","[Hello, doctor, ,, I, am, having, an, uncomfor...","[Hello, ., The, popping, and, discomfort, what..."
4,"[Q, ., My, symptoms, after, intercourse, threa...","[Hello, doctor, ,, Before, two, years, had, se...","[Hello, ., The, HIV, test, uses, a, finger, pr..."


No charts were generated by quickchart


**Lowercasing**

Let's convert our tokenized text to **lowercase** to ensure consistency in our data.

In [25]:
# Convert all tokens to lowercase
for col in ['Description_tokens', 'Patient_tokens', 'Doctor_tokens']:
    df[col] = df[col].apply(lambda x: [word.lower() for word in x])

df[['Description_tokens', 'Patient_tokens', 'Doctor_tokens']].head()


,Description_tokens,Patient_tokens,Doctor_tokens
0,"[q, ., what, does, abutment, of, the, nerve, r...","[hi, doctor, ,, i, am, just, wondering, what, ...","[hi, ., i, have, gone, through, your, query, w..."
1,"[q, ., what, should, i, do, to, reduce, my, we...","[hi, doctor, ,, i, am, a, 22-year-old, female,...","[hi, ., you, have, really, done, well, with, t..."
2,"[q., i, have, started, to, get, lots, of, acne...","[hi, doctor, !, i, used, to, have, clear, skin...","[hi, there, acne, has, multifactorial, etiolog..."
3,"[q, ., why, do, i, have, uncomfortable, feelin...","[hello, doctor, ,, i, am, having, an, uncomfor...","[hello, ., the, popping, and, discomfort, what..."
4,"[q, ., my, symptoms, after, intercourse, threa...","[hello, doctor, ,, before, two, years, had, se...","[hello, ., the, hiv, test, uses, a, finger, pr..."


**Stopword Removal**

By removing **Stopwords** like *the, is, and*   which don’t really  carry meaningful information we reduce the noise in our data.

In [ ]:
stop_words = set(stopwords.words('english'))

# Remove stopwords
for col in ['Description_tokens', 'Patient_tokens', 'Doctor_tokens']:
    df[col] = df[col].apply(lambda x: [word for word in x if word not in stop_words])

df[['Description_tokens', 'Patient_tokens', 'Doctor_tokens']].head()

**Punctuation Removal**

Removing punctuation marks like *!, ., ,*  because they rarely add semantic meaning in our text analyses.

In [26]:
# Remove punctuation
for col in ['Description_tokens', 'Patient_tokens', 'Doctor_tokens']:
    df[col] = df[col].apply(lambda x: [word for word in x if word not in string.punctuation])

df[['Description_tokens', 'Patient_tokens', 'Doctor_tokens']].head()


,Description_tokens,Patient_tokens,Doctor_tokens
0,"[q, what, does, abutment, of, the, nerve, root...","[hi, doctor, i, am, just, wondering, what, is,...","[hi, i, have, gone, through, your, query, with..."
1,"[q, what, should, i, do, to, reduce, my, weigh...","[hi, doctor, i, am, a, 22-year-old, female, wh...","[hi, you, have, really, done, well, with, the,..."
2,"[q., i, have, started, to, get, lots, of, acne...","[hi, doctor, i, used, to, have, clear, skin, b...","[hi, there, acne, has, multifactorial, etiolog..."
3,"[q, why, do, i, have, uncomfortable, feeling, ...","[hello, doctor, i, am, having, an, uncomfortab...","[hello, the, popping, and, discomfort, what, y..."
4,"[q, my, symptoms, after, intercourse, threatns...","[hello, doctor, before, two, years, had, sex, ...","[hello, the, hiv, test, uses, a, finger, prick..."


**Stemming and Lemmatization**

**Stemming** reduces words to their base forms e.g running to run.

**Lemmatization** refines this using linguistic context better to good.
Both this steps will help us unify variations of the same word.

In [28]:
## **Stemming and Lemmatization**
lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

df['Description'] = df['Description'].apply(lemmatize_tokens)
df['Patient'] = df['Patient'].apply(lemmatize_tokens)
df['Doctor'] = df['Doctor'].apply(lemmatize_tokens)
df.head()

,Description,Patient,Doctor,Description_tokens,Patient_tokens,Doctor_tokens
0,"[Q, ., , W, h, a, t, , d, o, e, s, , a, b, ...","[H, i, , d, o, c, t, o, r, ,, I, , a, m, , ...","[H, i, ., , I, , h, a, v, e, , g, o, n, e, ...","[q, what, doe, abut, of, the, nerv, root, mean]","[hi, doctor, i, am, just, wondering, what, is,...","[hi, i, have, gone, through, your, query, with..."
1,"[Q, ., , W, h, a, t, , s, h, o, u, l, d, , ...","[H, i, , d, o, c, t, o, r, ,, , I, , a, m, ...","[H, i, ., , Y, o, u, , h, a, v, e, , r, e, ...","[q, what, should, i, do, to, reduc, my, weight...","[hi, doctor, i, am, a, 22-year-old, female, wh...","[hi, you, have, really, done, well, with, the,..."
2,"[Q, ., , I, , h, a, v, e, , s, t, a, r, t, ...","[H, i, , d, o, c, t, o, r, !, , I, , u, s, ...","[H, i, , t, h, e, r, e, , A, c, n, e, , h, ...","[q., i, have, start, to, get, lot, of, acn, on...","[hi, doctor, i, used, to, have, clear, skin, b...","[hi, there, acne, has, multifactorial, etiolog..."
3,"[Q, ., , W, h, y, , d, o, , I, , h, a, v, ...","[H, e, l, l, o, , d, o, c, t, o, r, ,, I, , ...","[H, e, l, l, o, ., , T, h, e, , p, o, p, p, ...","[q, whi, do, i, have, uncomfort, feel, between...","[hello, doctor, i, am, having, an, uncomfortab...","[hello, the, popping, and, discomfort, what, y..."
4,"[Q, ., , M, y, , s, y, m, p, t, o, m, s, , ...","[H, e, l, l, o, , d, o, c, t, o, r, ,, B, e, ...","[H, e, l, l, o, ., , T, h, e, , H, I, V, , ...","[q, my, symptom, after, intercours, threatn, m...","[hello, doctor, before, two, years, had, sex, ...","[hello, the, hiv, test, uses, a, finger, prick..."


**Text Normalization**

Finally, we normalize the text by joining the processed tokens back into sentences.

In [30]:
# Rejoin tokens into cleaned text
for col in ['Description_tokens', 'Patient_tokens', 'Doctor_tokens']:
    df[col + '_cleaned'] = df[col].apply(lambda x: ' '.join(x))

df[['Description_tokens_cleaned', 'Patient_tokens_cleaned', 'Doctor_tokens_cleaned']].head()

,Description_tokens_cleaned,Patient_tokens_cleaned,Doctor_tokens_cleaned
0,q what doe abut of the nerv root mean,hi doctor i am just wondering what is abutting...,hi i have gone through your query with diligen...
1,q what should i do to reduc my weight gain due...,hi doctor i am a 22-year-old female who was di...,hi you have really done well with the hypothyr...
2,q. i have start to get lot of acn on my face p...,hi doctor i used to have clear skin but since ...,hi there acne has multifactorial etiology only...
3,q whi do i have uncomfort feel between the mid...,hello doctor i am having an uncomfortable feel...,hello the popping and discomfort what you felt...
4,q my symptom after intercours threatn me even ...,hello doctor before two years had sex with a c...,hello the hiv test uses a finger prick blood s...
